In [ ]:
import requests
import json
import pickle



def get_bne_wikidata(max_number=100):
    key = "bne"
    value = "wikidata"
    endpoint = """https://query.wikidata.org/bigdata/namespace/wdq/sparql?query="""
    
    query = """PREFIX wikibase: <http://wikiba.se/ontology>
    SELECT ?bne ?wikidata
    WHERE { 
        ?wikidata wdt:P950 ?bne .
    }
    LIMIT """+str(max_number)
    
    headers = {"Accept" : "application/json"}
    response = requests.get(endpoint+query, headers=headers)
    json_data = response.json()['results']['bindings']
    return [(item[key]['value'], item[value]['value']) for item in json_data]


datos = get_bne_wikidata()
#for key,value in datos:
    #print(key,"\t",value)

f = open("wikidatamap.bin", "wb+")
pickle.dump(datos, f)

In [ ]:
import importlib
import pickle
import dataset
importlib.reload(dataset)


datos = dataset.Dataset()
query2 = """PREFIX wikibase: <http://wikiba.se/ontology>
construct { ?wikidata ?predicate ?object . ?object ?predicate2 ?object2 . ?object2 ?predicate3 ?object3 }
WHERE { ?wikidata wdt:P950 ?bne .
?wikidata ?predicate ?object .
    ?object ?predicate2 ?object2 .
?object2 ?predicate3 ?object3
} LIMIT 10000 OFFSET 1880000
"""
datos.load_dataset_from_query(query2, only_uri=False)


#datos.load_dataset_from_nlevels(5, extra_params="LIMIT 100")
datos.show(verbose=False)
dataset1 = datos.train_split()

#     print(sub,dataset1[sub])
datos.save_to_binary("wikidata_2.bin")

# datos.subs

#print(len(jsonlist))
#jsonlist

In [ ]:
import numpy as np
rel = [i[2] for i in datos.subs]
print(len(datos.relations_dict))
print(len(datos.relations))
print(len(datos.entities_dict))
print(len(datos.entities))
rel = np.array(rel)
print(np.sort(rel))
data = np.matrix(datos.subs)
print(data[:,2])

print('Splitting the data into training and validation sets ...')
print(data.shape)
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
nb_validation_samples = int(0.2 * data.shape[0])


x_train = data[:-nb_validation_samples]
x_val = data[-nb_validation_samples:-int(nb_validation_samples/2)]
x_test = data[-int(nb_validation_samples/2):]

print(x_val.shape,"\n\n\n",x_test.shape,"\n\n\n",x_train.shape)
print(x_test.shape[0]+x_train.shape[0]+x_val.shape[0])


In [13]:
import importlib
import pickle
import dataset
importlib.reload(dataset)


datos = dataset.Dataset()
fb_test = open("FB15k/freebase_mtr100_mte100-test.txt", "r")
fb_train = open("FB15k/freebase_mtr100_mte100-train.txt", "r")
fb_valid = open("FB15k/freebase_mtr100_mte100-valid.txt", "r")
datos.load_dataset_from_csv(fb_train, "\t")
datos.load_dataset_from_csv(fb_test, "\t")
datos.load_dataset_from_csv(fb_valid, "\t")

datos.show(verbose=False)



14951 entities, 1345 relations, 592213 tripletas


In [14]:
en = dataset.Entities(datos).get_entity(50)
print(en)
id = dataset.Entities(datos).get_id("/m/063k3h")
print(id)
en2 = dataset.Entities(datos).get_entity(13299)
print(en2)
datos.entities.index("/m/063k3h")
datos.entities[50]

/m/063k3h
50
/m/0739z6


'/m/063k3h'

In [ ]:
import requests

def get_json_offset(n, count):
    endpoint = """https://query.wikidata.org/bigdata/namespace/wdq/sparql?query="""
    
    query = """PREFIX wikibase: <http://wikiba.se/ontology>
construct {{ ?wikidata ?predicate ?object . ?object ?predicate2 ?object2 . ?object2 ?predicate3 ?object3 }}
WHERE {{ ?wikidata wdt:P950 ?bne .
?wikidata ?predicate ?object .
?object ?predicate2 ?object2 .
?object2 ?predicate3 ?object3
}} LIMIT {0} OFFSET {1}
""".format(count, n*count)
    
    headers = {"Accept" : "application/json"}
    response = requests.get(endpoint+query, headers=headers)
    if response.status_code is not 200:
        return False, "Error occurred on http request. Code"+str(response.status_code)
    json_data = response.json()['results']['bindings']
    return json_data

all_json = []
for query in range(0,30):
    json1 = get_json_offset(query,20000)
    all_json = all_json + json1
    


In [ ]:
import importlib
import dataset
import queries
import pickle
importlib.reload(dataset)
importlib.reload(queries)

query = queries.Queries()

js = query.big_query(n_levels=3, n_rounds=4, total=10000)
print("finished. Saving")
f1 = open("millon.json", "wb+")
pickle.dump(js, f1)
f1.close()

In [ ]:
import importlib
import kgeserver.dataset as dataset
import kgeserver.wikidata_dataset as wikidata_dataset
import pickle
importlib.reload(dataset)
importlib.reload(wikidata_dataset)
from datetime import datetime

dtset = wikidata_dataset.WikidataDataset()


#dataset.load_entire_dataset(1)
dtset.load_dataset_recurrently(3, verbose=2, limit_ent=500)
dtset.show()



In [22]:
print(dtset.relations[8])
dtset.save_to_binary("wikidata_25k.bin")

P31


True

In [ ]:
import importlib
import dataset
import algorithm
import skge
importlib.reload(dataset)
importlib.reload(algorithm)


dt = dataset.Dataset()

dt.load_from_binary("wn18.bin")

al = algorithm.Algorithm(dt)

models = models = al.find_best(ncomps=[50], margins=[2], model_types=[skge.TransE])

In [ ]:
import importlib
import dataset
import algorithm
import skge
importlib.reload(dataset)
importlib.reload(algorithm)


dt = dataset.Dataset()

dt.load_from_binary("wn18.bin")

al = algorithm.Algorithm(dt)

models = models = al.find_best(ncomps=[150], margins=[0.2], model_types=[skge.HolE])

In [52]:
import requests

def extract_triples_from_statement(uri):
    st_query = """PREFIX wikibase: <http://wikiba.se/ontology>
      SELECT ?pred ?subj
      WHERE {{
      <{0}> ?pred ?subj .
      }}""".format(uri)
    response = requests.get("https://query.wikidata.org/bigdata/namespace/wdq/sparql?query="+st_query, headers={"Accept": "application/json"})

    el_json = response.json()["results"]["bindings"]

    # Check errors
#     if sts is not 200:
#         return None
    el_queue = []
    
    for el in el_json:
        pr = (el['pred']['value'])        
        sj = (el['subj']['value'])
        sj = check_entity(sj)
        pr = check_relation(pr)
        if sj:
            el_queue.append(sj)

   
    return el_queue
extract_triples_from_statement("http://www.wikidata.org/entity/statement/q180-BD4C0870-A565-4952-BF6E-F435F7ACF615")


None None
None None
None None
P159 Q62
None None
None None
None None
None None
['Q62']


['Q62']

In [47]:
def check_entity(entity):
    """Check the entity given and return a valid representation

    :param string entity: The input entity representation
    :return: A valid representation or None
    :rtype: string
    """
    # This expects as input an entity URI
    # http://www.wikidata.org/entity/Q180
    try:
        # If either fails to convert the last Q number into int
        # or the URI hasn't 'entity' keyword, returns without doing nothing
        ent_uri = entity.split("/")
        wikidata_id = ent_uri[-1]
        # print(entity, wikidata_id)
        if ent_uri[-2] == 'entity' and type(int(wikidata_id[1:])) is int and wikidata_id[0] is "Q":
            return wikidata_id
        else:
            return None
    except Exception:
        return None
    return None

def check_relation( relation):
    """Check the relation given and return a valid representation

    :param string relation: The input relation representation
    :return: A valid representation or None
    :rtype: string
    """
    # http://www.wikidata.org/prop/direct/P2853 VALID
    # http://www.wikidata.org/prop/statement/P159 VALID
    # http://www.wikidata.org/prop/qualifier/P18 NOT VALID
    try:
        # If either fails to convert the last Q number into int
        # or the URI hasn't 'entity' keyword, returns without doing nothing
        prp_uri = relation.split("/")
        wikidata_prop = prp_uri[-1]
        if wikidata_prop[0] is "P" and prp_uri[-3] == 'prop' and (prp_uri[-2] == "direct" or prp_uri[-2] == "statement") and type(int(wikidata_prop[1:])) is int:
            return wikidata_prop
        else:
            return None
    except Exception:
        print("ex")
        return None
    return None
print(check_relation("http://www.wikidata.org/prop/direct/P2853"))
print(check_relation("http://www.wikidata.org/prop/statement/P159"))
print(check_relation("http://www.wikidata.org/prop/qualifier/P18"))
print(check_entity("http://www.wikidata.org/value/db2cef2363893c6df38812b8c3b38263"))
print(check_entity("http://www.wikidata.org/entity/Q62"))
print(check_entity("http://commons.wikimedia.org/wiki/Special:FilePath/New%20Wikimedia%20Foundation%20Office%2014.jpg/Q34"))

P2853
P159
None
None
Q62
None
